In [1]:
from lxml import html

# from bs4 import BeautifulSoup
from bs4 import BeautifulSoup, NavigableString
import time
import datetime
import threading
from apscheduler.schedulers.background import BackgroundScheduler

from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager

from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

from selenium.webdriver.chrome.options import Options

from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as ec

from selenium.common.exceptions import TimeoutException

import helper

import json
import os
import sys

In [2]:
course_schedule = []

IMPLICIT_WAIT_TIME = 20
EXPLICIT_WAIT_TIME = 20


RETRY_INTERVAL = 65

In [3]:
from selenium.common.exceptions import NoSuchElementException        
def check_exists_by_xpath(xpath):
    try:
        driver.find_element("xpath",xpath)
    except NoSuchElementException:
        return False
    return True

def sync_get_element_by_xpath(driver, wait, xpath):
    try:
        wait.until(ec.visibility_of_element_located((By.XPATH, xpath)))
    except TimeoutException:
        print("Get element from xpath:{} Timeout".format(xpath))
        return None
    return driver.find_element("xpath",xpath)

In [4]:
def teacher_norm(teacher):
    if not teacher.startswith('-'):
        return teacher
    tmp = teacher.split('\n')[0].split('- ')[1]
    return tmp[:-2]

def search_courses_by_date(date_str, courses):
    results = []
    
    date_str = date_str.strip()
    
    for item in courses:
        if date_str in item['date']:
            results.append(item)
    return results

def search_courses_by_teacher(teacher_str, courses):
    results = []
    
    teacher_str = teacher_str.strip()
    
    for item in courses:
        if teacher_str in item['teacher']:
            results.append(item)
    return results

def search_courses_by_name(name_str, courses):
    results = []
    
    name_str = name_str.strip()
    
    for item in courses:
        if name_str in item['name']:
            results.append(item)
    return results

def search_courses_by_time(time_str, courses):
    results = []
    
    time_str = time_str.strip()
    
    for item in courses:
        if time_str in item['time']:
            results.append(item)
    return results

    
def list_courses_to_be_booked(courses_to_be_booked):
    cur_date = ''
    if len(courses_to_be_booked) <= 0:
        return
    
    for course in courses_to_be_booked:  

        if cur_date != course['date']:
            if cur_date != '':
                print("-----------------------")
            print("Date: {}".format(course['date']))
            print("-----------------------\n")
            cur_date = course['date']
        print("Name: {}".format(course['name']))
        print("Teacher: {}".format(course['teacher']))
        print("Time: {}".format(course['time']))
        print("\n")
        

In [5]:
def create_new_driver():
    #start web driver

    from selenium import webdriver
    from selenium.webdriver.chrome.service import Service

    service = Service(executable_path=r'/usr/bin/chromedriver')

    #setup option for chrome profile
    options = webdriver.ChromeOptions()
    options.add_argument('--headless')
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')

    driver = webdriver.Chrome(service=service, options=options)

    driver.implicitly_wait(IMPLICIT_WAIT_TIME)
    wait = WebDriverWait(driver, EXPLICIT_WAIT_TIME)
    

    return driver, wait


In [6]:
def send_keys(elm, string):
    for c in string:
        elm.send_keys(c)
        
def sign_in(driver, wait):
    
    username = sync_get_element_by_xpath(driver, wait, "//input[@id='ctl00_cphContents_txtUsername']")
    password = sync_get_element_by_xpath(driver, wait, "//input[@id='ctl00_cphContents_txtPassword']")
    if username is not None:
        send_keys(username, "DN20092360")
    if password is not None:
        send_keys(password, "jj1216")

#     username.send_keys("DN23035242")
#     password.send_keys("NTW3BSFH")

    
    try:
        wait.until(ec.visibility_of_element_located((By.CLASS_NAME, "submit-button")))
    except TimeoutException:
        print("Get element with class name of submit-button imeout")
        return None
    
    sign_in_btn = sync_get_element_by_xpath(driver, wait, "//div[@class='submit-button']")
    sign_in_btn.click()

def log_out(driver, wait):
    click_btn = sync_get_element_by_xpath(driver, wait, "//a[contains(text(), 'Logout')]")
#     print("click_btn: ", click_btn)
    if click_btn:
        click_btn.click()
    
def click_book_for_class(driver, wait):
    click_btn = sync_get_element_by_xpath(driver, wait, "//a[contains(text(), '預定課')]")

#     print("click_btn: ", click_btn)
    click_btn.click()
    
def click_book_this_class_now(driver, wait):

    click_btn = sync_get_element_by_xpath(driver, wait, "//a[contains(text(), '現在就預定此課')]")

    if click_btn:
        click_btn.click()
        return click_btn


In [7]:
#launch url
base_url = "https://www.trueclassbooking.com.tw/member/search-class.aspx"

In [8]:
temp = []

# load schedule from file
##---------------------------------------------------------------
with open("./class_schedules/27 十一月 - 03 十二月 2023.json", "r") as read_file:
    course_schedule = json.load(read_file)

Day_string = "28"

c = helper.search_courses_by_date(Day_string, course_schedule)
c = helper.search_courses_by_name('Yin', c)
c = helper.search_courses_by_teacher('Mar', c)
c = helper.search_courses_by_time('', c)

temp.extend(c)


##---------------------------------------------------------------


courses_to_be_booked = temp

list_courses_to_be_booked(courses_to_be_booked)

Date: 28 11月 周二
-----------------------

Name: Yin Yang Yoga 陰陽瑜珈
Teacher: Marcus
Time: 05:45 - 06:45 下午




In [9]:
master_counter = 1

In [10]:
def sign_in_once(driver, wait):
#     print("sign_in({},{})".format(driver, wait))
    driver.get(base_url)
    sign_in(driver, wait)

In [11]:
def timestamp_string():

    date_time = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
#     date_time = datetime.datetime.now().strftime("%S")

    return date_time

In [12]:
csv_file_name = "./log_try_booking_a_class_that_was_full-- {}.csv".format(timestamp_string())

In [13]:
class_idx = 1
num_classes = len(courses_to_be_booked)

In [14]:
def reserve_class(driver, wait, course_to_be_booked, idx):
    
    global master_counter
    global class_idx
    global num_classes

    stride =RETRY_INTERVAL / 60 * 15 * num_classes

    book_url = course_to_be_booked['href']
    class_name = course_to_be_booked['name']
    teacher = course_to_be_booked['teacher']
    

    driver.get(book_url)

    click_btn=click_book_this_class_now(driver, wait)
    if click_btn is not None:
        
        print("master counter = {}, stride = {}, class_idx = {}".format(master_counter, stride, class_idx))

        log_str = "'{}','{}' by '{}',BOOKED,'{}'\n".format(master_counter, class_name, teacher, datetime.datetime.now())
        print(log_str)
        
        #CSV file table header
        import os
        if not os.path.exists(csv_file_name):
            with open(csv_file_name, "w") as write_file:
                write_file.write("#,Class,Status,Time\n")

        with open(csv_file_name, "a") as write_file:
            write_file.write(log_str)

        num_classes = num_classes - 1
    else:
        
        print("master counter = {}, stride = {}, class_idx = {}".format(master_counter, stride, class_idx))

        next_book_datetime = datetime.datetime.now() + datetime.timedelta(seconds=RETRY_INTERVAL)
        log_str = "'{}','{}' by '{}',FULL,'{}'\nnext_book_datetime={}\nclick_btn={}\n".format(master_counter, class_name, teacher, datetime.datetime.now(), next_book_datetime, click_btn)
        print(log_str)
        
        if 0==0: #(master_counter - class_idx) % stride == 0:
            
            #CSV file table header
            import os
            if not os.path.exists(csv_file_name):
                with open(csv_file_name, "w") as write_file:
                    write_file.write("#,Class,Status,Time\n")

            with open(csv_file_name, "a") as write_file:
                write_file.write(log_str)
       
                
        driver.close()
        
        driver, wait = create_new_driver()

        sign_in_once(driver, wait)
        next_book_datetime = datetime.datetime.now() + datetime.timedelta(seconds=60+(idx%len(course_to_be_booked))*10 )
        print("now={}".format(datetime.datetime.now()))
        print("next_book_datetime={}".format(next_book_datetime))
        scheduler.add_job(reserve_class, 'date', run_date= next_book_datetime, args=[driver, wait, course_to_be_booked, idx])
    
        master_counter = master_counter + 1
    
    if class_idx < num_classes:
        class_idx = class_idx + 1
    else:
        class_idx = 1


In [15]:
drivers = []
waits = []
scheduler = BackgroundScheduler()
start = time.time()
        
for idx, course_to_be_booked in enumerate(courses_to_be_booked):

    driver, wait = create_new_driver()
    
    sign_in_once(driver, wait)
    next_book_datetime = datetime.datetime.now() + datetime.timedelta(seconds=60+(idx%len(course_to_be_booked))*10 )
    print("now={}".format(datetime.datetime.now()))
    print("next_book_datetime={}".format(next_book_datetime))
    scheduler.add_job(reserve_class, 'date', run_date= next_book_datetime, args=[driver, wait, course_to_be_booked, idx])
    
# print("Time Elapsed: {:.2f}".format(time.time() - start)) 
scheduler.start()

now=2023-11-27 18:26:17.745903
next_book_datetime=2023-11-27 18:27:17.745874


Get element from xpath://a[contains(text(), '現在就預定此課')] Timeout
master counter = 1, stride = 16.25, class_idx = 1
'1','Yin Yang Yoga 陰陽瑜珈' by 'Marcus',FULL,'2023-11-27 18:27:38.464924'
next_book_datetime=2023-11-27 18:28:43.464905
click_btn=None

now=2023-11-27 18:27:44.360681
next_book_datetime=2023-11-27 18:28:44.360661
Get element from xpath://a[contains(text(), '現在就預定此課')] Timeout
master counter = 2, stride = 16.25, class_idx = 1
'2','Yin Yang Yoga 陰陽瑜珈' by 'Marcus',FULL,'2023-11-27 18:29:05.059397'
next_book_datetime=2023-11-27 18:30:10.059376
click_btn=None

now=2023-11-27 18:29:10.567685
next_book_datetime=2023-11-27 18:30:10.567666
Get element from xpath://a[contains(text(), '現在就預定此課')] Timeout
master counter = 3, stride = 16.25, class_idx = 1
'3','Yin Yang Yoga 陰陽瑜珈' by 'Marcus',FULL,'2023-11-27 18:30:31.296762'
next_book_datetime=2023-11-27 18:31:36.296740
click_btn=None

now=2023-11-27 18:30:37.010616
next_book_datetime=2023-11-27 18:31:37.010597
Get element from xpath://a[c